# Comparison of NMF and QMF on cancer datasets

We reproduce the second sub-benchmark proposed in the [MOMIX benchmarck](https://github.com/ComputationalSystemsBiology/momix-notebook) to compare the performances of NMF and QMF on the integration of multi-omics cancer datasets from TCGA. The methods are evaluated regarding associations to survival.

## Comparison based on survival predictions

We here define the function that compares the performances of NMF and QMF based on their ability to infer factors associated to survival (computed through Cox regression).

In [1]:
library('survival')

## Perform survival annotation-based comparison 
## INPUTS:
# factorizations = already computed factorizations
# method = methods used for factorization
# survival = survival data associated to the cancer
# out.folder = folder where results will be written
# cancer = name of currently analysed cancer
## OUPUTS: a list containing output values
# The function also plots P-values of association, as in the original MOMIX paper
survival_comparison_merge <- function(factorizations, method, survival, out.folder, cancer){
    
    # Initialize result containers
    factors_cancer <- numeric(0)
    surv_final <- numeric(0)
    
    # For each computed factorisation
    for(i in 1:length(factorizations)){

        # Extract sample factors 
        factors <- factorizations[[i]][[1]]

        # Patient names in factorisation results
        patient.names <- substr(rownames(factors),1,12) # extract just the first 12 characters of patient ID
        # Patient names in original data
        patient.names.in.file <- substr(as.character(survival[, 1]),1,12) # extract just the first 12 characters of patient ID
        patient.names.in.file <- toupper(gsub('-', '\\.', patient.names.in.file))
        # Remove non-matching patient names
        is_in_file <- patient.names %in% patient.names.in.file
        if(length(patient.names)!=sum(is_in_file)) {
            factors <- factors[is_in_file, ]
            patient.names <- patient.names[is_in_file]
            rownames(factors)<-patient.names
        }
        # Match indices of patient names
        indices <- match(patient.names, patient.names.in.file)
        # Use indices to extract coresponding survival information
        ordered.survival.data <- survival[indices,]
        # Clean data (assign 0 to NAs)
        ordered.survival.data$Survival[is.na(ordered.survival.data$Survival)] <- 0
        ordered.survival.data$Death[is.na(ordered.survival.data$Death)] <- 0

        # Calculate coxph
        coxph_obj <- coxph(Surv(ordered.survival.data$Survival, ordered.survival.data$Death) ~ factors)
        # P-values (corrected by the number of methods)
#        pvalues <- length(factorizations)*as.matrix(coef(summary(coxph_obj))[,5])
        pvalues <- as.matrix(coef(summary(coxph_obj))[,5])

        # How many significant? 
        factors_cancer <- c(factors_cancer, sum(pvalues<0.05))
        # Store p-values
        surv_final <- cbind(surv_final, sort(pvalues)) 
    }
    surv_merge <- numeric(0)
    umethod <- unique(method)
    for (m in umethod) {
        col <- which(method %in% m)
        replicates <- surv_final[,col,drop=FALSE]
        sorted_replicates <- matrix(sort(replicates),nrow=length(col))
        surv_merge <- cbind(surv_merge, apply(sorted_replicates,2,mean))
    }

    # Keep -log10 of p-values
    surv_final<-(-log10(surv_merge))
    surv_final_red <- surv_final
    surv_final_red[surv_final <= -log10(0.05)] <- NA
    # Plot survival pvalues for each cancer type separately
    png(file=paste0(out.folder, "survival_", cancer, ".png"), width = 3, height = 5, units = 'in', res = 200)
    par(mar = c(2, 4, 2, 0)+0.1)
    matplot(1:length(umethod), t(surv_final), 
            col="black", pch=18, xlab="", ylab="-log(corrected P-value survival)", main=cancer, cex=1.5, ylim=c(0, 2.5), xlim=c(0.5, 2.5), xaxt="none")
    matplot(1:length(umethod), t(surv_final_red), 
            col="red", pch=18, xaxt="none", cex=1.5, add=T)
    abline(h = (-log10(0.05)), v=0, col="black", lty=3, lwd=3)
    axis(1, at=1:length(umethod), labels=umethod) 
    dev.off()
    
    # Same plot, without y axis
    png(file=paste0(out.folder, "survival_", cancer, "_noyaxis.png"), width = 2, height = 5, units = 'in', res = 200)
    par(mar = c(2, 0, 2, 0)+0.1)
    matplot(1:length(umethod), t(surv_final), 
            col="black", pch=18, xlab="", ylab="Pvalues survival", main=cancer, cex=1.5, ylim=c(0, 2.5), xlim=c(0.5, 2.5), xaxt="none")
    matplot(1:length(umethod), t(surv_final_red), 
            col="red", pch=18, xaxt="none", cex=1.5, add=T)
    abline(h = (-log10(0.05)), v=0, col="black", lty=3, lwd=3)
    axis(1, at=1:length(umethod), labels=umethod) 
    dev.off()

    return(surv_final)
}

## Running the comparisons in cancer

Here we run the comparison on the cancer data. The NMF and QMF factorizations have been computed offline, and the resulting factors are stored in the data/factorizations directory.

In [2]:
# Function to read the factorizations of NMF and QMF
readmydata <- function(cancer, rpath, numfact=10) {
    factorizations<-list()
    method <- c()
    methodpath <- c()

    listdir <- list.dirs(rpath)
    listdir <- listdir[grepl(paste0('dataset=',cancer), listdir, fixed=TRUE)]
    listdir <- listdir[grepl('/results', listdir, fixed=TRUE)]
    
    isnmf <- grepl('NMF', listdir, fixed=TRUE)
    isqmf <- grepl('@Suquan', listdir, fixed=TRUE)
    islog <- grepl('to_log2=exp,methy', listdir, fixed=TRUE)


    ipath <- which(isnmf & islog)
    if (length(ipath)>0) {
        for (i in ipath) {
            method <- c(method, "NMF")
            methodpath <- c(methodpath, listdir[i])
        }
    }

    ipath <- which(isqmf & islog & grepl('regularization_factor=0.001:', listdir, fixed=TRUE))
    if (length(ipath)>0) {
        for (i in ipath) {
            method <- c(method, "QMF")
            methodpath <- c(methodpath, listdir[i])
        }    
    }

    t<-1
    for (mypath in methodpath) {
        myv <- as.matrix(read.table(paste0(mypath, "/v.csv"), sep=",", header=FALSE, row.names = 1))
        colnames(myv) <- 1:numfact
        myu1 <- as.matrix(read.table(paste0(mypath, "/u_exp.csv"), sep=",", header=FALSE, row.names = 1))
        colnames(myu1) <- 1:numfact
        myu2 <- as.matrix(read.table(paste0(mypath, "/u_methy.csv"), sep=",", header=FALSE, row.names = 1))
        colnames(myu1) <- 1:numfact
        myu3 <- as.matrix(read.table(paste0(mypath, "/u_mirna.csv"), sep=",", header=FALSE, row.names = 1))
        colnames(myu1) <- 1:numfact
        
        factorizations[[t]]<-list(myv, list(myu1, myu2, myu3))
        if (max(abs(myv))+max(abs(myu1))+max(abs(myu2))+max(abs(myu3)) > 1e10) {
            print(paste('Warning: large values in ',mypath))
        }

        t <- t+1
    }
    return(list(factorizations=factorizations,method=method))
}

In [3]:
# List cancer data.
# Exclude first result as it's the parent folder
cancers <- list.dirs(path = "../data/cancer", full.names = TRUE, recursive = TRUE)[-1]
# We have no results for ovarian
cancers <- cancers[-grep('ovarian',cancers,fixed=TRUE)]

# Label to identify current run
tag <- format(Sys.time(), "%Y%m%d%H%M%S")
# Folder for comparison results
results_folder <- paste0("../results", tag, "/")
# Create output folder
dir.create(results_folder, showWarnings = FALSE)

# Number of factors used in the paper
num.factors <- 10

for(i in cancers){

    print(paste0("Now analysing ", i))
    
    # Name of current cancer
    current_cancer <- basename(i)

    # Perform factorisation
    out <- readmydata(current_cancer, '../data/factorizations')

    # Survival analysis
    survival <- read.table(paste0(i, "/survival"), sep="\t", header=TRUE)
    out_survival <- survival_comparison_merge(out$factorizations, out$method, survival, 
                                        results_folder, current_cancer)
}    


Warning message in as.POSIXlt.POSIXct(x, tz):
“unknown timezone 'zone/tz/2019c.1.0/zoneinfo/Europe/Paris'”


[1] "Now analysing ../data/cancer/aml"


Warning message in min(x):
“aucun argument trouvé pour min ; Inf est renvoyé”
Warning message in max(x):
“aucun argument pour max ; -Inf est renvoyé”
Warning message in min(x):
“aucun argument trouvé pour min ; Inf est renvoyé”
Warning message in max(x):
“aucun argument pour max ; -Inf est renvoyé”


[1] "Now analysing ../data/cancer/breast"
[1] "Now analysing ../data/cancer/colon"


Warning message in min(x):
“aucun argument trouvé pour min ; Inf est renvoyé”
Warning message in max(x):
“aucun argument pour max ; -Inf est renvoyé”
Warning message in min(x):
“aucun argument trouvé pour min ; Inf est renvoyé”
Warning message in max(x):
“aucun argument pour max ; -Inf est renvoyé”


[1] "Now analysing ../data/cancer/gbm"


Warning message in min(x):
“aucun argument trouvé pour min ; Inf est renvoyé”
Warning message in max(x):
“aucun argument pour max ; -Inf est renvoyé”
Warning message in min(x):
“aucun argument trouvé pour min ; Inf est renvoyé”
Warning message in max(x):
“aucun argument pour max ; -Inf est renvoyé”


[1] "Now analysing ../data/cancer/kidney"


Warning message in min(x):
“aucun argument trouvé pour min ; Inf est renvoyé”
Warning message in max(x):
“aucun argument pour max ; -Inf est renvoyé”
Warning message in min(x):
“aucun argument trouvé pour min ; Inf est renvoyé”
Warning message in max(x):
“aucun argument pour max ; -Inf est renvoyé”


[1] "Now analysing ../data/cancer/liver"
[1] "Now analysing ../data/cancer/lung"
[1] "Now analysing ../data/cancer/melanoma"
[1] "Now analysing ../data/cancer/sarcoma"
